In [1]:
import os, sys
lib_path = os.path.abspath(os.path.join('..'))
sys.path.append(lib_path)

In [2]:
import unittest
from unittest.mock import patch
from ddt import ddt, data, unpack
from script.consulscanner_hash import ConsulScannerHash

In [3]:
@ddt
class ConsulScannerHashTestCase(unittest.TestCase):
    def setUp(self):
        self.myConsulScannerHash = ConsulScannerHash("mssql", "192.168.7.134", 1433, "SA", "Helios12$", "huaPIITest")

    def tearDown(self):
        self.myConsulScannerHash.conn.close()
        self.myConsulScannerHash = None

    @data(
        ("192.168.7.5", 8500, "mssql_uuid_2", 'mssql_alias', "helios_scanner_uuid", 'mssql_tenant_uuid', "pii_1st", "uid", "pii_2nd", ["firstname", "lastname", "gender"]),
        ("192.168.7.5", 8500, "mssql_uuid_3", 'mssql_alias', "helios_scanner_uuid", 'mssql_tenant_uuid', "pii_1st", "uid", "pii_3rd", ["phonenumber", "email"]),
        ("192.168.7.5", 8500, "mssql_uuid_4", 'mssql_alias', "helios_scanner_uuid", 'mssql_tenant_uuid', "pii_1st", "uid", "pii_4th", ["country", "city", "stateprovince"])
    )
    @unpack
    def test_consulkv_put_scanner_source(self,
                                         consul_ip, consul_port,
                                         sourceid, alias, scannerid, tenantid,
                                         keytabname, keycolname, tabname, colnames):
        with patch('script.consulscanner_hash.ConsulScannerHash.consulkv_put_scanner_source', return_value=1) as mock_consulkv_put_scanner_source:
            res = mock_consulkv_put_scanner_source(consul_ip, consul_port,
                sourceid, alias, scannerid, tenantid,
                keytabname, keycolname, tabname, colnames)
            self.assertEqual(res, 1)

    @data(("192.168.7.5", 8500, "pii_2nd", "firstname"),
          ("192.168.7.5", 8500, "pii_2nd", "lastname"),
          ("192.168.7.5", 8500, "pii_2nd", "gender"),
          ("192.168.7.5", 8500, "pii_3rd", "phonenumber"),
          ("192.168.7.5", 8500, "pii_3rd", "email"),
          ("192.168.7.5", 8500, "pii_4th", "country"),
          ("192.168.7.5", 8500, "pii_4th", "city"),
          ("192.168.7.5", 8500, "pii_4th", "stateprovince"))
    @unpack
    def test_get_scanner_hashed(self, consul_ip, consul_port, tabname, colname):
        res = self.myConsulScannerHash.consulkv_get_scanner_hashed(consul_ip, consul_port, tabname, colname)
        self.assertTrue(res)
        self.assertIsInstance(res, list)
        self.assertIsInstance(res[0], str)
        
if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)

test_consulkv_put_scanner_source_1___192_168_7_5___8500___mssql_uuid_2____mssql_alias____helios_scanner_uuid____mssql_tenant_uuid____pii_1st____uid____pii_2nd_____firstname____lastname____gender___ (__main__.ConsulScannerHashTestCase) ... ok
test_consulkv_put_scanner_source_2___192_168_7_5___8500___mssql_uuid_3____mssql_alias____helios_scanner_uuid____mssql_tenant_uuid____pii_1st____uid____pii_3rd_____phonenumber____email___ (__main__.ConsulScannerHashTestCase) ... ok
test_consulkv_put_scanner_source_3___192_168_7_5___8500___mssql_uuid_4____mssql_alias____helios_scanner_uuid____mssql_tenant_uuid____pii_1st____uid____pii_4th_____country____city____stateprovince___ (__main__.ConsulScannerHashTestCase) ... ok
test_get_scanner_hashed_1___192_168_7_5___8500___pii_2nd____firstname__ (__main__.ConsulScannerHashTestCase) ... ok
test_get_scanner_hashed_2___192_168_7_5___8500___pii_2nd____lastname__ (__main__.ConsulScannerHashTestCase) ... ok
test_get_scanner_hashed_3___192_168_7_5___8500___pii_

### End of this file